To be honest, I don't think this code is working really well, because the simulations look very weird... (you can find them in the folder gifs_lucia)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio

In [2]:
it = 0

In [27]:
it += 1
N = 50
timesteps = 500 # 50 seconds
boundaries = 100

#parameters are: zor, zoo, zoa, alpha, theta, s, sigma, tau
params = [20, 20, 30, 300, 50, 3, 5, 0.1]
params_string = f'it={it}, zor={params[0]}, zoo={params[1]}, zoa={params[2]}, alpha={params[3]}, theta={params[4]}, s={params[5]}, sigma={params[6]}, tau={params[7]}'

init = np.random.rand(N, 4)

In [28]:
def r_ij(arr, i, j):
    '''returns unit vector from individual i in direction of individual j'''
    diff = arr[j, :2]-arr[i, :2]
    return diff/np.linalg.norm(diff)

def ZOR(arr, i, params, boundaries, check_speed=True):
    if check_speed:
        s = params[5]
        tau = params[7]
        r_r = params[0]
        assert s*tau<r_r
    l = []
    for j in range(N):
        if j != i:
            diff = arr[j, :2]-arr[i, :2]
            if boundaries[0]<=np.linalg.norm(diff)<boundaries[1]: # make sure that fish is inside zor
                R = r_ij(arr, i, j)/np.linalg.norm(r_ij(arr, i, j))
                l.append(R)
    s = -sum(l)
    return s/np.linalg.norm(s), len(l)
    
def ZOO(arr, i, params):
    r_r = params[0]
    r_o = params[1]
    l = []
    for j in range(N):
        diff = arr[j, -2:]-arr[i, -2:]
        if r_r<=np.linalg.norm(diff)<r_o: # make sure that fish is inside zoo
            R = arr[j, -2:]/np.linalg.norm(arr[j, -2:])
            l.append(R)
    s = sum(l)
    return s/np.linalg.norm(s), len(l)

def ZOA(arr, i, params, boundaries, check_speed=False):
    return -ZOR(arr, i, params, boundaries)[0], ZOR(arr, i, params, boundaries)[1]
    

In [29]:
arr = init
arrays = []
for timestep in range(timesteps):
    arr_new = arr.copy()
    for i in range(N):
        zor, n_r = ZOR(arr, i, params, (0, params[0]))
        zoo, n_o = ZOO(arr, i, params)
        zoa, n_a = ZOA(arr, i, params, (params[1], params[2]))

        if n_r>0:
            vel = zor

        elif n_o>0:
            if n_a==0:
                vel = zoo
            else:
                vel = (1/2)*(zoo+zoa)

        elif n_a>0:
            if n_o==0:
                vel = zoa
            else:
                vel = (1/2)*(zoo+zoa)

        if vel.all()==0:
            vel = arr[i, -2:]
        
        # periodic boundary conditions 
        if abs(arr[i, 0]) >= boundaries or abs(arr[i, 1]) >= boundaries:
            arr[i, 0] = -arr[i, 0]
            arr[i, 1] = -arr[i, 1]

        arr_new[i] = [arr[i, 0]+arr[i, 2], arr[i, 1]+arr[i, 3], vel[0], vel[1]]
    
    arrays.append(arr_new)
    arr = arr_new
    

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


In [30]:
arrays[2]

array([[-1.60238266, -0.04241585, -0.87785895, -0.47891927],
       [ 2.15332222, -1.16577858,  0.59001853, -0.8073897 ],
       [ 2.76469695,  0.4449793 ,  0.99582262,  0.09130889],
       [-0.40568423, -0.02994947, -0.24588455, -0.96929912],
       [-1.28816755, -0.91774196, -0.75162763, -0.65958768],
       [ 2.38709394, -0.57747078,  0.54710559, -0.8370636 ],
       [-1.44197165,  0.02281527, -0.98894371, -0.14829142],
       [ 0.55975744,  3.08106825, -0.44513185,  0.89546504],
       [-0.76577823,  1.95151154, -0.56417747,  0.82565355],
       [ 3.31950853,  1.44362662,  0.99881637, -0.04864018],
       [ 2.3266242 , -1.51743568,  0.44510062, -0.89548056],
       [ 2.73338982,  3.57910466,  0.5921763 ,  0.80580843],
       [ 2.08897067,  3.06282591,  0.63189027,  0.77505786],
       [-0.26857601, -1.44258011, -0.49515442, -0.86880498],
       [ 1.98151831,  0.12746177,  0.11935943, -0.99285111],
       [ 0.61762614,  2.26231841,  0.64647405,  0.76293598],
       [ 3.55099373, -0.

In [31]:
png_list = []
for i in range(timesteps):
    x = arrays[i][:, 0] 
    y = arrays[i][:, 1]
    u = arrays[i][:, 2]
    v = arrays[i][:, 3]
    fig = plt.figure(figsize=(10, 10))
    plt.quiver(x, y, u, v) # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html
    plt.xlim(-boundaries, boundaries)
    plt.ylim(-boundaries, boundaries)
    plt.savefig(f'plots_lucia/{i}.png')
    png_list.append(f'plots_lucia/{i}.png')
    plt.close()


In [32]:
with imageio.get_writer('gifs_lucia/'+params_string+'.gif', mode='I') as writer:
    for filename in png_list:
        image = imageio.imread(filename)
        writer.append_data(image)